<a href="https://colab.research.google.com/github/massimopiazza/colmap-gs/blob/main/COLMAP-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing COLMAP on Google Colab

COLMAP is a 3D reconstruction software, that runs Feature Detection & Matching, Structure-from-Motion (SfM), and Multi-View Stereo (MVS).

## Install Dependencies & set up Drive access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!sudo apt update
!sudo apt install -y \
    build-essential cmake git \
    libboost-all-dev libeigen3-dev libsuitesparse-dev \
    qtbase5-dev libglew-dev libglfw3-dev \
    libx11-dev libopencv-dev libgoogle-glog-dev \
    libgflags-dev libatlas-base-dev libopencv-core-dev \
    libopenimageio-dev openimageio-tools libopenexr-dev \
    libcgal-dev libcgal-qt5-dev libmetis-dev

### Install Abseil (Ceres dependency)
Abseil is a collection of open-source C++ libraries from Google. Ceres depends on Abseil for some of its internal utilities (e.g. containers, flags, threading helpers).

In [2]:
%cd /content
!git clone https://github.com/abseil/abseil-cpp.git
%cd /content/abseil-cpp
!git checkout 20230802.0

!rm -rf build
!mkdir build
%cd build
!cmake .. \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local \
  -DCMAKE_POSITION_INDEPENDENT_CODE=ON
!make -j$(nproc)
!sudo make install

/content
Cloning into 'abseil-cpp'...
remote: Enumerating objects: 33992, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 33992 (delta 151), reused 95 (delta 89), pack-reused 33715 (from 4)
Receiving objects: 100% (33992/33992), 19.51 MiB | 17.76 MiB/s, done.
Resolving deltas: 100% (26554/26554), done.
/content/abseil-cpp
Note: switching to '20230802.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 29bf8085 Cherry pick CCTZ fix for MinGW 

### Install Ceres Solver (using the just installed Abseil)
Ceres is an open-source C++ library for non-linear least squares optimization. COLMAP uses Ceres to optimize camera poses, 3D points, and bundle adjustment.

In [ ]:
%cd /content
!git clone https://github.com/ceres-solver/ceres-solver.git
%cd /content/ceres-solver
!git checkout 2.1.0

!rm -rf build
!mkdir build
%cd build
!cmake .. \
  -DBUILD_TESTING=OFF \
  -DBUILD_EXAMPLES=OFF \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local
!make -j$(nproc)
!sudo make install

/content
Cloning into 'ceres-solver'...
remote: Enumerating objects: 23212, done.
remote: Counting objects: 100% (570/570), done.
remote: Compressing objects: 100% (364/364), done.
remote: Total 23212 (delta 336), reused 206 (delta 206), pack-reused 22642 (from 4)
Receiving objects: 100% (23212/23212), 18.19 MiB | 8.54 MiB/s, done.
Resolving deltas: 100% (16119/16119), done.
/content/ceres-solver
Note: switching to '2.1.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at f68321e7 Update version history
/content

## Install COLMAP (w/ CUDA)

In [ ]:
if False:
  %cd /content
  !git clone https://github.com/colmap/colmap.git
  %cd colmap

  !rm -rf build
  !mkdir build
  %cd build
  !cmake .. \
      -DCMAKE_BUILD_TYPE=Release \
      -DCMAKE_INSTALL_PREFIX=/usr/local \
      -DCUDA_ENABLED=ON \
      -DCeres_DIR=/usr/local/lib/cmake/Ceres \
      -DAbsl_DIR=/usr/local/lib/cmake/absl

  !make -j$(nproc)
  !sudo make install

  ## test COLMAP installation
  !colmap --help

### COLMAP cached build (if available)

In [ ]:
%%bash
set -euo pipefail
set -x
export DEBIAN_FRONTEND=noninteractive

LOG="/content/colmap-build.log"
: > "$LOG"
exec > >(stdbuf -oL -eL tee -a "$LOG") 2>&1
echo "Logging to: $LOG"
date

################### UNCOMMENT TO FORCE CACHE CLEANUP ###########################
#rm -rf "/content/drive/MyDrive/COLMAP/colab-install-cache/ccache"
#rm -rf "/content/drive/MyDrive/COLMAP/colab-install-cache/git-mirror"
#rm -rf "/content/drive/MyDrive/COLMAP/colab-install-cache/prefix"
#rm -f  "/content/drive/MyDrive/COLMAP/colab-install-cache/build_signature.txt"
################################################################################

# -------- user-configurable knobs --------
COLMAP_REPO_URL="https://github.com/colmap/colmap.git"
COLMAP_GIT_REF="${COLMAP_GIT_REF:-main}"          # e.g. "master", a tag, or a commit SHA
FORCE_REBUILD="${FORCE_REBUILD:-0}"    # set to 1 to ignore cache #@param {type:"string"}
CUDA_ENABLED="${CUDA_ENABLED:-ON}"

# These are your existing dependency locations (as in your snippet)
CERES_DIR="${CERES_DIR:-/usr/local/lib/cmake/Ceres}"
ABSL_DIR="${ABSL_DIR:-/usr/local/lib/cmake/absl}"

# Pulled from the Python cell:
COLMAP_CACHE_ROOT="${COLMAP_CACHE_ROOT:-/content/drive/MyDrive/COLMAP/colab-install-cache}" #@param {type:"string"}

# -------- derived paths --------
INSTALL_DIR="/content/colmap_install"              # fast local disk for runtime use
SRC_DIR="/content/colmap_src"
BUILD_DIR="/content/colmap_build"

mkdir -p "$COLMAP_CACHE_ROOT"
META_FILE="$COLMAP_CACHE_ROOT/meta_${COLMAP_GIT_REF}.json"
TARBALL="$COLMAP_CACHE_ROOT/colmap_install_${COLMAP_GIT_REF}.tar.zst"

# ---- ccache that works with Google Drive mount ----
sudo apt-get update -y
sudo apt-get install -y ccache ninja-build zstd

# Persistent cache storage (Drive)
export CCACHE_DIR="$COLMAP_CACHE_ROOT/ccache"
mkdir -p "$CCACHE_DIR"

# Temp files MUST be on local disk (Drive/FUSE doesn't like hardlinks)
export CCACHE_TEMPDIR="/content/ccache-tmp"
mkdir -p "$CCACHE_TEMPDIR"

# Be explicit: do NOT use hard links or reflinks
export CCACHE_NOHARDLINK=1
export CCACHE_NOFILECLONE=1

# Optional but nice
export CCACHE_MAXSIZE=10G

# Reset stats (optional)
ccache -z || true
ccache -p | sed -n '1,120p'

# Detect GPU compute capability (for safe reuse when using CMAKE_CUDA_ARCHITECTURES=native)
GPU_CC="nogpu"
if command -v nvidia-smi >/dev/null 2>&1; then
  # Example output: "8.0" -> store "80"
  GPU_CC="$(nvidia-smi --query-gpu=compute_cap --format=csv,noheader | head -n1 | tr -d '.' || true)"
  [ -n "$GPU_CC" ] || GPU_CC="unknown"
fi

# We’ll default to "native" for fastest build, but cache is only valid for same GPU_CC
CUDA_ARCH="${CUDA_ARCH:-native}"   # alternatives: "all-major", "all", or "75" etc. :contentReference[oaicite:3]{index=3}

# -------- cache validity check --------
CACHE_OK=0
if [[ "$FORCE_REBUILD" == "0" && -f "$TARBALL" && -f "$META_FILE" ]]; then
  # validate metadata quickly (git ref + cuda arch + gpu cc)
  python3 - <<PY || exit 0
import json, os, sys
meta=json.load(open("${META_FILE}"))
want = {
  "git_ref": "${COLMAP_GIT_REF}",
  "cuda_enabled": "${CUDA_ENABLED}",
  "cuda_arch": "${CUDA_ARCH}",
  "gpu_cc": "${GPU_CC}",
  "ceres_dir": "${CERES_DIR}",
  "absl_dir": "${ABSL_DIR}",
}
ok = all(meta.get(k)==v for k,v in want.items())
sys.exit(0 if ok else 1)
PY
  if [[ $? -eq 0 ]]; then CACHE_OK=1; fi
fi

# -------- restore fast path --------
if [[ "$CACHE_OK" == "1" ]]; then
  echo "✅ Restoring cached COLMAP install from Drive tarball:"
  echo "   $TARBALL"

  rm -rf "$INSTALL_DIR"
  mkdir -p "$INSTALL_DIR"
  tar -I zstd -xf "$TARBALL" -C /content

else
  echo "🛠️  Cache miss (or incompatible) -> building COLMAP from source."

  rm -rf "$SRC_DIR" "$BUILD_DIR" "$INSTALL_DIR"
  git clone --recursive "$COLMAP_REPO_URL" "$SRC_DIR"
  cd "$SRC_DIR"
  git checkout "$COLMAP_GIT_REF"
  git submodule update --init --recursive

  COLMAP_COMMIT="$(git rev-parse HEAD)"
  echo "Building commit: $COLMAP_COMMIT"

  # Note: Some setups require CUDA libs on LD_LIBRARY_PATH when detected via legacy CMake module. :contentReference[oaicite:4]{index=4}
  export LD_LIBRARY_PATH="/usr/local/cuda/lib64:${LD_LIBRARY_PATH:-}"

  cmake -S "$SRC_DIR" -B "$BUILD_DIR" -G Ninja \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_INSTALL_PREFIX="$INSTALL_DIR" \
    -DCUDA_ENABLED=ON \
    -DGUI_ENABLED=OFF \
    -DCeres_DIR="$CERES_DIR" \
    -DAbsl_DIR="$ABSL_DIR" \
    -DCMAKE_C_COMPILER_LAUNCHER=ccache \
    -DCMAKE_CXX_COMPILER_LAUNCHER=ccache \
    -DCMAKE_EXE_LINKER_FLAGS="-Wl,--no-as-needed -lGL -lGLX -lOpenGL -lGLdispatch -Wl,--as-needed"

  # Reduce parallel jobs to avoid potential memory/CPU issues during compilation
  ninja -C "$BUILD_DIR" -j$(($(nproc) > 2 ? $(nproc)/2 : 1))
  ninja -C "$BUILD_DIR" install

  # Save metadata + tarball cache back to Drive
  python3 - <<PY
import json, time
meta = {
  "built_at": time.strftime("%Y-%m-%d %H:%M:%S"),
  "git_ref": "${COLMAP_GIT_REF}",
  "git_commit": "${COLMAP_COMMIT}",
  "cuda_enabled": "${CUDA_ENABLED}",
  "cuda_arch": "${CUDA_ARCH}",
  "gpu_cc": "${GPU_CC}",
  "ceres_dir": "${CERES_DIR}",
  "absl_dir": "${ABSL_DIR}",
}
with open("${META_FILE}", "w") as f: json.dump(meta, f, indent=2)
print("Wrote:", "${META_FILE}")
PY

  echo "📦 Creating Drive cache tarball (this is what makes next sessions fast)…"
  # Tar the *installed* prefix (not build dir) to avoid thousands of object files on Drive
  rm -f "$TARBALL"
  tar -I "zstd -19 -T0" -cf "$TARBALL" -C /content "$(basename "$INSTALL_DIR")"
  echo "Saved:", "$TARBALL"

fi

# -------- activate COLMAP for this session --------
export PATH="$INSTALL_DIR/bin:$PATH"
export LD_LIBRARY_PATH="$INSTALL_DIR/lib:$INSTALL_DIR/lib64:${LD_LIBRARY_PATH:-}"

# Sanity check
echo "colmap path: $(which colmap)"
colmap --help | head -n 20

# Show ccache stats (useful to see hits on rebuilds)
ccache -s || true


# Run COLMAP (GPU) on your images
Folder structure for your images have to be like this:

```
/content/project/
├── images/        # your input images
├── database.db
└── sparse/

```

You can put images in your Google Drive, ```image_source``` folder, and change the path below. This command will copy images from the ```image_source``` folder on your Google Drive into your Colab environment.


In [ ]:
import datetime
import os
import shutil
from google.colab import drive
import subprocess # Added this import

drive.mount('/content/drive')
%cd /content

img_source_dir = '/content/drive/MyDrive/Gaussian-Splatting/images/croatia-house' #@param {type:"string"}

# Define global variables for project structure
database_name = 'database.db'
images_folder_name = 'images'
sparse_folder_name = 'sparse'
undistorted_folder_name = 'undistorted'
dense_folder_name = 'dense'
sparse_model_name = '0' # Folder created by COLMAP mapper within sparse_folder_name
fused_ply_name = 'fused.ply'
meshed_poisson_ply_name = 'meshed-poisson.ply'

# Default GPU index to use if GPU is available
GPU_DEFAULT_INDEX = 0

# Create timestamped project folder
ts = datetime.datetime.now().isoformat(timespec='seconds').replace(':', '')
image_folder_name_from_path = os.path.basename(img_source_dir)
project_root = f'/content/{image_folder_name_from_path}_{ts}'

# Construct full paths
images_path = os.path.join(project_root, images_folder_name)
database_path = os.path.join(project_root, database_name)
sparse_path = os.path.join(project_root, sparse_folder_name)
undistorted_path = os.path.join(project_root, undistorted_folder_name)
dense_path = os.path.join(project_root, dense_folder_name)
sparse_model_path = os.path.join(sparse_path, sparse_model_name)
fused_ply_path = os.path.join(dense_path, fused_ply_name)
meshed_poisson_ply_path = os.path.join(dense_path, meshed_poisson_ply_name)


print(f'Creating new project at: {project_root}')
os.makedirs(images_path, exist_ok=True)

# Set COLMAP environment variables for subsequent shell commands
os.environ['PATH'] = f'/content/colmap_install/bin:{os.environ.get("PATH", "")}'
os.environ['LD_LIBRARY_PATH'] = f'/content/colmap_install/lib:/content/colmap_install/lib64:{os.environ.get("LD_LIBRARY_PATH", "")}'

# Detect GPU availability
GPU_AVAILABLE = False
try:
    # Attempt to run nvidia-smi. If it succeeds, a GPU is likely available.
    subprocess.check_output(['nvidia-smi'])
    GPU_AVAILABLE = True
    print("GPU is available and will be used for COLMAP operations.")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("No GPU detected or nvidia-smi command not found. COLMAP operations will run on CPU.")

# Copy images
if os.path.exists(img_source_dir):
    !cp -r "{img_source_dir}" "{images_path}"
else:
    print(f'Warning: Source directory {img_source_dir} not found. Skipped copying.')

### Start Feature Extraction

If your images are from different cameras, set ```ImageReader.single_camera``` to ```0```, else set to ```1```.

In [ ]:
!export QT_QPA_PLATFORM=offscreen
!colmap feature_extractor \
  --database_path "{database_path}" \
  --image_path "{images_path}" \
  --ImageReader.single_camera 0 \
  --FeatureExtraction.use_gpu {int(GPU_AVAILABLE)}

### Feature matching (exhaustive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!colmap exhaustive_matcher \
  --database_path "{database_path}" \
  --FeatureMatching.use_gpu {int(GPU_AVAILABLE)}

### Sparse reconstruction

In [ ]:
!mkdir -p "{sparse_path}"
!colmap mapper \
  --database_path "{database_path}" \
  --image_path "{images_path}" \
  --output_path "{sparse_path}"

### Export undistorted COLMAP reconstruction

In [ ]:
!colmap image_undistorter \
    --image_path "{images_path}" \
    --input_path "{sparse_model_path}" \
    --output_path "{undistorted_path}" \
    --output_type COLMAP \
    --max_image_size 1600

## Dense reconstruction (+ meshing)

In [ ]:
# --------- dense MVS workspace ----------
!mkdir -p "{dense_path}"
!colmap image_undistorter \
  --image_path "{images_path}" \
  --input_path "{sparse_model_path}" \
  --output_path "{dense_path}" \
  --output_type COLMAP \
  --max_image_size 1600

# --------- dense depth maps (GPU) ----------
!colmap patch_match_stereo \
  --workspace_path "{dense_path}" \
  --workspace_format COLMAP \
  --PatchMatchStereo.gpu_index 0 \
  --PatchMatchStereo.geom_consistency true \
  --PatchMatchStereo.cache_size 10

# --------- fuse to dense point cloud ----------
!colmap stereo_fusion \
  --workspace_path "{dense_path}" \
  --output_path "{fused_ply_path}" \
  --workspace_format COLMAP \
  --input_type geometric \
  --StereoFusion.cache_size 10 \
  --StereoFusion.num_threads $(nproc)

# --------- Poisson mesh ----------
!colmap poisson_mesher \
  --input_path "{fused_ply_path}" \
  --output_path "{meshed_poisson_ply_path}" \
  --PoissonMeshing.trim 5 \
  --PoissonMeshing.point_weight 0.5 \
  --PoissonMeshing.num_threads $(nproc)
#trim  = discard signal value on mesh lower than threshold;
#        ypical tuning is 0 (discard the least), 2, 5, 10 (default value), 20
#weight= 0 --> more smoothing/filling where surface points not detected
#        1 (default)
#        4 (stick more to point cloud, but gets too spiky if PC is noisy)

### Copy COLMAP results to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p "/content/drive/MyDrive/COLMAP/output"
!cp -r "{project_root}" "/content/drive/MyDrive/COLMAP/output"